In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Introduction

We start by opening the data and creating a pandas dataframe with all the information form the dataset. We set the "Start Date" field as the index to make data manipulation easier.

In [ ]:
df = pd.read_csv("../data/EVChargingStationUsage.csv")
df["Start Date"] = pd.to_datetime(df["Start Date"])
df["End Date"] = pd.to_datetime(df["End Date"], errors="coerce")
df["Charging Time (hh:mm:ss)"] = pd.to_timedelta(df["Charging Time (hh:mm:ss)"])
df = df.set_index(df["Start Date"])

We will only use fields described below for the task of predicting consumption. 

In [ ]:
df = df[["Station Name", "Energy (kWh)", "Charging Time (hh:mm:ss)"]]
df

## Data cleaning and reformating

Now we have to reformat the data so that instead of displaying consumption for each time a car was charged on a station, we want have aggrigated data with total consumption for the whole hour for each of the stations. We will use 2 assumptions to get this: The first is a constant consumption throughout the whole charging period and the second one is that every car starts charging immediately after arriving at the charging station. The second assumption is needed since we do not know the exact time of charging start and finish, but it seems reasonable to assume that people start charging their cars right after arriving at the charging station, but they might stay at the station longer than is needed for the car to charge completely for reasons. For example, if the start date is 00:00, end date 03:00 and charging duration is 2 hours, and charging consumed 10kWh of energy, we will assume the consumption between both 00:00 to 01:00 and 01:00 to 02:00 is 5kWh, and 0kWh between 02:00 and 03:00. In other words, we ignore the end data for this prediction.

To do this, we will create a function which takes in a dataframe for one station and creates another dataframe with hourly consumption data.

In [ ]:
def get_hourly_consumption(df: pd.DataFrame) -> pd.Series:
    first_datetime = df.index.min().replace(minute=0, second=0)
    last_datetime = (df.index.max() + df.iloc[-1]["Charging Time (hh:mm:ss)"]).replace(minute=0, second=0)
    hourly_index = pd.date_range(first_datetime, last_datetime, freq="H")
    hourly_df = pd.Series(0.0, index=hourly_index)
    for index, row in df.iterrows():
        current_datetime = index
        total_consumption = row["Energy (kWh)"]
        total_duration = row["Charging Time (hh:mm:ss)"]
        end_datetime = index + total_duration
        # print(str(index) + " - " + str(end_datetime))
        while current_datetime < end_datetime:
            start_hour = current_datetime.replace(minute=0, second=0)
            end_hour = start_hour + pd.Timedelta(hours=1)
            sub_duration = min(end_hour, end_datetime) - current_datetime
            sub_consumption = total_consumption * sub_duration / total_duration
            hourly_df[start_hour] += sub_consumption
            current_datetime = end_hour
    return hourly_df

In [ ]:
station_names = df['Station Name'].unique()
len(station_names)

There are a total of 47 stations in the dataset. Lets create a dictionary with station names as keys and hourly energy consumption dataframe as values.

In [ ]:
hourly_consumption_by_station = dict()
for i, station_name in enumerate(station_names):
    station_df = df[df["Station Name"] == station_name]
    consumption = get_hourly_consumption(station_df)
    hourly_consumption_by_station[station_name] = consumption


## Seasonality and trend visualisation

Now we plot 3 graphs for 2 arbitrarily chosen stations to observe seasonality and trends in our data.

In [ ]:
def plot_consumption(hourly_consumption: pd.Series):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    plot_day_consumption(hourly_consumption, axes[0])
    plot_week_consumption(hourly_consumption, axes[1])
    plot_month_consumption(hourly_consumption, axes[2])
    plt.show()

def plot_day_consumption(hourly_consumption: pd.Series, ax: plt.Axes):
    hourly_grouped = hourly_consumption.groupby(hourly_consumption.index.hour)
    data = [hourly_grouped.get_group(hour).values for hour in range(24) if hour in hourly_grouped.groups]
    ax.boxplot(data, positions=range(len(data)), widths=0.6, patch_artist=True, showmeans=True, showfliers=False)
    ax.set_xlabel("Hour of the day")
    ax.set_ylabel("Average consumption (kWh)")
    ax.set_title("Average consumption per hour of the day")

def plot_week_consumption(hourly_consumption: pd.Series, ax: plt.Axes):
    # Group data by day of the week
    weekly_data = [
        hourly_consumption[hourly_consumption.index.dayofweek == day].values
        for day in range(7)
    ]
    
    # Draw the boxplot
    ax.boxplot(weekly_data, positions=range(7), widths=0.6, patch_artist=True, showmeans=True, showfliers=False)
    ax.set_xlabel("Day of the week")
    ax.set_ylabel("Consumption (kWh)")
    ax.set_title("Distribution of Consumption per Day of the Week")
    ax.set_xticks(range(7))
    ax.set_xticklabels(["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"])


def plot_month_consumption(hourly_consumption: pd.Series, ax: plt.Axes):
    # Group data by month
    monthly_data = [
        hourly_consumption[hourly_consumption.index.month == month].values
        for month in range(1, 13)
    ]
    
    # Draw the boxplot
    ax.boxplot(monthly_data, positions=range(1, 13), widths=0.6, patch_artist=True, showmeans=True, showfliers=False)
    ax.set_xlabel("Month")
    ax.set_ylabel("Consumption (kWh)")
    ax.set_title("Distribution of Consumption per Month")
    ax.set_xticks(range(1, 13))
    ax.set_xticklabels([
        "Jan", "Feb", "Mar", "Apr", "May", "Jun",
        "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"
    ])


In [ ]:
plot_consumption(hourly_consumption_by_station["PALO ALTO CA / CAMBRIDGE #1"])
plot_consumption(hourly_consumption_by_station["PALO ALTO CA / HIGH #1"])
plot_consumption(hourly_consumption_by_station["PALO ALTO CA / RINCONADA LIB 1"])

The day consumption seems to be greater than the night consumption. But on the other hand, week and month patterns don't seem to match. Interestingly, the third station seems to behave differently than first 2, having more evenly distributed hourly consumption through the day, higher weekend consumption and no spikes during January and February.

Now lets visualize the trends in the data for the same 3 stations. We used daily consumption to make the graphs more readable.

In [ ]:
def plot_trends(hourly_consumption: pd.Series):
    daily_consumption = hourly_consumption.resample("D").sum()
    weekly_window = daily_consumption.rolling(window=7).mean()
    yearly_window = daily_consumption.rolling(window=365).mean()

    # Plot daily, 7-day rolling mean, and 365-day rolling mean time series
    fig, ax = plt.subplots(figsize=(15, 5))
    ax.plot(daily_consumption, marker='.', markersize=2, color='0.6',
    linestyle='None', label='Daily')
    ax.plot(weekly_window, linewidth=2, label='7-d Rolling Mean')
    ax.plot(yearly_window, color='0.2', linewidth=3,
    label='Trend (365-d Rolling Mean)')

    ax.legend()
    ax.set_xlabel('Year')
    ax.set_ylabel('Consumption (kWh)')
    ax.set_title('Charging Station Consumption Trend')

In [ ]:
plot_trends(hourly_consumption_by_station["PALO ALTO CA / CAMBRIDGE #1"])
plot_trends(hourly_consumption_by_station["PALO ALTO CA / HIGH #1"])
plot_trends(hourly_consumption_by_station["PALO ALTO CA / RINCONADA LIB 1"])

In all graphs there seem to be 2 sudden drops of consumption: in the middle of year 2017 and at the beginning of 2020.

## Autocorrelation and partial autocorrelation

Before building any model, we need to see how many datapoints from the past should we use in our autoregressive model. We can use plot_acf and plot_pacf methods to help us decide.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def plot_autocorrelations(consumption: pd.Series):
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    plot_acf(consumption, ax=axs[0])
    plot_pacf(consumption, ax=axs[1])
    plt.show()

In [ ]:
plot_autocorrelations(hourly_consumption_by_station["PALO ALTO CA / CAMBRIDGE #1"])
plot_autocorrelations(hourly_consumption_by_station["PALO ALTO CA / HIGH #1"])
plot_autocorrelations(hourly_consumption_by_station["PALO ALTO CA / RINCONADA LIB 1"])

The graphs suggest that using past 3 hours along with the same hour of the previous day (indicated by the peak at 24 hour delay in the graphs) might yield the best model.

## Stationarity

The last step before building a model is determining stationarity. Looking at the graphs in the trend section, the data does not seem to stationary, but we can confirm that with a ADF test.

In [ ]:
from statsmodels.tsa.stattools import adfuller

def print_adf_results(consumption: pd.Series):
    result = adfuller(consumption)
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

We will perform the test on both hourly and daily data

In [ ]:
print_adf_results(hourly_consumption_by_station["PALO ALTO CA / CAMBRIDGE #1"])
print_adf_results(hourly_consumption_by_station["PALO ALTO CA / HIGH #1"])
print_adf_results(hourly_consumption_by_station["PALO ALTO CA / RINCONADA LIB 1"])

In [ ]:
print_adf_results(hourly_consumption_by_station["PALO ALTO CA / CAMBRIDGE #1"].resample("D").sum())
print_adf_results(hourly_consumption_by_station["PALO ALTO CA / HIGH #1"].resample("D").sum())
print_adf_results(hourly_consumption_by_station["PALO ALTO CA / RINCONADA LIB 1"].resample("D").sum())

Interestingly, the hourly data seems to be stationary while daily data does not. The reason might be that there is a long term trend (over years) which hourly test can't detect. So, just to make sure, we will use first order differencing in our model.

## Creating training and test data

In [ ]:
import pandas as pd

test_size = 0.3
training_data = dict()
test_data = dict()

for station_name, hourly_df in hourly_consumption_by_station.items():
    # Calculate the split index based on test size
    split_index = int(len(hourly_df) * (1 - test_size))
    
    # Get the timestamp corresponding to the split index
    split_timestamp = hourly_df.index[split_index]
    
    # Adjust the split index to the next midnight (start of the day)
    adjusted_split_index = hourly_df.index.get_loc(
        hourly_df[hourly_df.index >= split_timestamp.normalize()].index[0]
    )
    
    # Perform the split
    training_split = hourly_df.iloc[:adjusted_split_index]
    test_split = hourly_df.iloc[adjusted_split_index:]
    
    # Remove the last day's data from the test split
    if not test_split.empty:
        last_day_start = test_split.index[-1].normalize()
        test_split = test_split[test_split.index < last_day_start]
    
    # Assign splits to dictionaries
    training_data[station_name] = training_split
    test_data[station_name] = test_split

# Example: Accessing test data for a specific station
test_data["PALO ALTO CA / CAMBRIDGE #1"]


## Evaluation

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error

def print_stats(y_test, y_preds, title="You forgot the title!"):
    print(title+": ")
    print("\tr^2=%f"%r2_score(y_test,y_preds))
    print("\tMAE=%f"%mean_absolute_error(y_test,y_preds))

## Baseline model

We decided to use historical averages for each hour of the day in the week for our baseline model.

In [ ]:
def plot_predicted_vs_actual(y_test, y_preds, title="You forgot the title!"):
    plt.figure(figsize=(10,5))
    plt.plot(y_test.index[:500], y_test[:500], label="Actual")
    plt.plot(y_test.index[:500], y_preds[:500], label="Predicted")
    plt.title(title)
    plt.legend(["Actual", "Predicted"])
    plt.show()

In [ ]:
# Filter for each day of the week
days_in_week = 7
daily_distributions = []
for day in range(days_in_week):
    day_of_week_data = training_data["PALO ALTO CA / CAMBRIDGE #1"][training_data["PALO ALTO CA / CAMBRIDGE #1"].index.dayofweek == day]
    daily_distributions.append(day_of_week_data.groupby(day_of_week_data.index.hour).mean())

# Create predictions
predictions = []
for i in range(len(test_data["PALO ALTO CA / CAMBRIDGE #1"])):
    day_of_week = test_data["PALO ALTO CA / CAMBRIDGE #1"].index[i].dayofweek
    hour = test_data["PALO ALTO CA / CAMBRIDGE #1"].index[i].hour
    predictions.append(daily_distributions[day_of_week][hour])

# Calculate and print stats and plot results
print_stats(test_data["PALO ALTO CA / CAMBRIDGE #1"].values, predictions, "PALO ALTO CA / CAMBRIDGE #1")
plot_predicted_vs_actual(test_data["PALO ALTO CA / CAMBRIDGE #1"], predictions, "PALO ALTO CA / CAMBRIDGE #1")

## XGB model

In [ ]:
from xgboost import XGBRegressor

# Function to create lag features
lags = [1, 2, 3, 24, 48, 72, 168, 336, 504]
def create_lag_features(df: pd.DataFrame) -> pd.DataFrame:
    for lag in lags:
        df[f"lag_{lag}"] = df["Consumption"].shift(lag)
    return df

lagged_df_training = create_lag_features(pd.DataFrame(training_data["PALO ALTO CA / CAMBRIDGE #1"], columns=["Consumption"]))
lagged_df_training = lagged_df_training.dropna().reset_index(drop=True)

# Splitting into features and target
X_train = lagged_df_training.drop(columns=["Consumption"])
y_train = lagged_df_training["Consumption"]

targets = test_data["PALO ALTO CA / CAMBRIDGE #1"]

model = XGBRegressor(n_estimators=1000, learning_rate=0.01, max_depth=10, random_state=42)
model.fit(X_train, y_train)

In [ ]:
targets

In [ ]:
# Dynamic forecasting: Predict the entire next day at midnight
# Initialize with the most recent data available from the training/test sets
last_known_data = training_data["PALO ALTO CA / CAMBRIDGE #1"].iloc[-max(lags):]  # Latest 504 hours (for lags)

predictions = pd.Series()  # Store all predictions
i = 0
for dt in test_data["PALO ALTO CA / CAMBRIDGE #1"].index:
    new_row = {f"lag_{lag}": last_known_data.iloc[-lag] for lag in lags}
    prediction = model.predict(pd.DataFrame(new_row, index=[0]))
    predictions[dt] = prediction[0]
    last_known_data = pd.concat([last_known_data.iloc[1:], pd.Series(prediction[0], index=[dt])])

    if dt.hour == 23:
        # i+=1
        last_known_data = pd.concat([last_known_data.iloc[:-24], pd.Series(targets[targets.index.normalize() == dt.normalize()])])
        # if i == 100:
        #     break

# Evaluate predictions against actual test data
# targets = targets.iloc[:100*24]
print_stats(targets, predictions, "XGBoost")
plot_predicted_vs_actual(targets, predictions, "XGBoost")

## SARIMAX model

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA

# Define the seasonal period for daily seasonality in hourly data
seasonal_period = 24  # 24 hours per day

# Training the SARIMA model on the training data
model = SARIMAX(training_data["PALO ALTO CA / CAMBRIDGE #1"].iloc[:1000], order=(3, 1, 0), seasonal_order=(3, 1, 0, seasonal_period))
# model = ARIMA(training_data["PALO ALTO CA / CAMBRIDGE #1"].iloc[:10000], order=(3, 1, 3))
ARIMA_model = model.fit()
print(ARIMA_model.summary())

# Forecasting with the SARIMA model, appending each new observation without refitting
preds = []
for l in range(0, 50):#len(test_data["PALO ALTO CA / CAMBRIDGE #1"].iloc[:500])):
    preds.append(ARIMA_model.forecast().iloc[0])  # Get a one-step forecast
    next_observation = test_data["PALO ALTO CA / CAMBRIDGE #1"].iloc[l]
    
    # Append the next observation without refitting the model
    ARIMA_model = ARIMA_model.append([next_observation], refit=False)

plt.plot(test_data["PALO ALTO CA / CAMBRIDGE #1"].iloc[:50].values, label="Actual")
plt.plot(preds, label="Predicted")
plt.legend(["Actual", "Predicted"])
plt.show()
print_stats(test_data["PALO ALTO CA / CAMBRIDGE #1"].iloc[:50].values, preds, "PALO ALTO CA / CAMBRIDGE #1")
